相较于5272，少了一个last_ts2（少了一个时序窗口为2的滞后值），且模型lr从0.06调整至0.08

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from matplotlib.pyplot import plot, show
from sklearn.metrics import roc_auc_score
import json
from gensim.models.word2vec import Word2Vec

df_train = pd.read_csv('data/train_dataset.csv',sep='\t')
df_test = pd.read_csv('data/test_dataset.csv', sep='\t')
sub = pd.read_csv('data/submit_example.csv')
df_test['id'] = sub['id']
df = pd.concat([df_train, df_test])

In [2]:
df['location_first_lvl'] = df['location'].astype(str).apply(lambda x: json.loads(x)['first_lvl'])
df['location_sec_lvl'] = df['location'].astype(str).apply(lambda x: json.loads(x)['sec_lvl'])
df['location_third_lvl'] = df['location'].astype(str).apply(lambda x: json.loads(x)['third_lvl'])

feats = ['user_name', 'action', 'auth_type', 'ip_location_type_keyword', 'ip_risk_level', 'ip', 'location',
         'device_model', 'os_type', 'os_version', 'browser_type', 'browser_version', 'bus_system_code', 'op_target',
         'location_first_lvl', 'location_sec_lvl', 'location_third_lvl',
         ]

print(len(feats))

17


In [3]:
cat = []

LABEL = 'risk_label'

In [4]:
# 特征处理

df['sec'] = df.session_id.apply(lambda x:int(x[-7:-5]))
df['sec_sin'] = np.sin(df['sec']/60*2*np.pi)
df['sec_cos'] = np.cos(df['sec']/60*2*np.pi)

df['op_date'] = pd.to_datetime(df['op_date'])
df['hour'] = df['op_date'].dt.hour
df['weekday'] = df['op_date'].dt.weekday
df['year'] = df['op_date'].dt.year
df['month'] = df['op_date'].dt.month
df['day'] = df['op_date'].dt.day
# 转Unix时间戳
df['op_ts'] = df["op_date"].values.astype(np.int64) // 10 ** 9

# 根据用户、时间戳排序
df = df.sort_values(by=['user_name', 'op_ts']).reset_index(drop=True)

df['last_ts'] = df.groupby(['user_name'])['op_ts'].shift(1)
df['last_ts2'] = df.groupby(['user_name'])['op_ts'].shift(2)
df['ts_diff'] = df['op_ts'] - df['last_ts']
df['ts_diff2'] = df['op_ts'] - df['last_ts2']

In [5]:
feats += ['sec',
          'sec_sin', 'sec_cos',
          'op_ts', 'last_ts', 'ts_diff',
          # 'last_ts2',
          'ts_diff2',
          ]

In [6]:
for name in ['auth_type']:
    df[name+'_fillna'] = df[name].astype('str')
    sent = df.groupby(['user_name', 'year', 'month', 'day'])[name+'_fillna'].agg(list).values

    vec_size = 6
    w2v_model = Word2Vec(sentences=sent, vector_size=vec_size, window=12, min_count=1, workers=1)
    tmp = df[name+'_fillna'].map(lambda x: w2v_model.wv[x])
    tmp = pd.DataFrame(list(tmp))
    tmp.columns = ['_'.join([name, 'emb', str(i)]) for i in range(vec_size)]
    df = pd.concat([df, tmp], axis=1)
    feats += list(tmp.columns)

In [7]:
for w in w2v_model.wv.key_to_index:
    print(w, w2v_model.wv[w])

nan [-0.13903026  0.03005757  0.25148448  0.9644866  -0.28805935 -0.67375904]
pwd [ 0.03973053  0.1891723   0.13216272  1.0011268  -0.16735148 -0.7792582 ]
sms [-0.12732314  0.05897151  0.15256187  1.0156738  -0.19783318 -0.69883054]
qr [-0.15083304 -0.18690944  0.32928893  1.0530428  -0.14471556 -0.6634975 ]
otp [ 0.10661024 -0.01189896  0.15297233  0.95404774 -0.40966824 -0.6917451 ]


In [8]:
for name in ['mean', 'std', 'max', 'min', 'median', 'skew']:
    for name1 in ['user_name', 'bus_system_code', 'auth_type', 'action',
                  ]:  # 'op_target'

        df[name1+'_ts_diff_'+name] = df.groupby([name1])['ts_diff'].transform(name)
        feats.append(name1+'_ts_diff_'+name)

In [9]:
df['if_out'] = (df['location'] == '{"first_lvl":"成都分公司","sec_lvl":"9楼","third_lvl":"销售部"}')
feats.append('if_out')

In [10]:
for name in ['user_name', 'action', 'auth_type', 'ip', 'ip_location_type_keyword', 'ip_risk_level', 'location',
             'device_model', 'os_type', 'os_version', 'browser_type', 'browser_version', 'bus_system_code', 'op_target',
             'location_first_lvl', 'location_sec_lvl', 'location_third_lvl',
             ]+cat:
    le = LabelEncoder()
    df[name] = le.fit_transform(df[name])


In [11]:
df_train = df[~df[LABEL].isna()].reset_index(drop=True)
df_test = df[df[LABEL].isna()].reset_index(drop=True)

In [12]:
params = {
    'learning_rate': 0.08,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
}

In [13]:
print(feats)
print(df_train[feats].shape, df_test[feats].shape)

['user_name', 'action', 'auth_type', 'ip_location_type_keyword', 'ip_risk_level', 'ip', 'location', 'device_model', 'os_type', 'os_version', 'browser_type', 'browser_version', 'bus_system_code', 'op_target', 'location_first_lvl', 'location_sec_lvl', 'location_third_lvl', 'sec', 'sec_sin', 'sec_cos', 'op_ts', 'last_ts', 'ts_diff', 'ts_diff2', 'auth_type_emb_0', 'auth_type_emb_1', 'auth_type_emb_2', 'auth_type_emb_3', 'auth_type_emb_4', 'auth_type_emb_5', 'user_name_ts_diff_mean', 'bus_system_code_ts_diff_mean', 'auth_type_ts_diff_mean', 'action_ts_diff_mean', 'user_name_ts_diff_std', 'bus_system_code_ts_diff_std', 'auth_type_ts_diff_std', 'action_ts_diff_std', 'user_name_ts_diff_max', 'bus_system_code_ts_diff_max', 'auth_type_ts_diff_max', 'action_ts_diff_max', 'user_name_ts_diff_min', 'bus_system_code_ts_diff_min', 'auth_type_ts_diff_min', 'action_ts_diff_min', 'user_name_ts_diff_median', 'bus_system_code_ts_diff_median', 'auth_type_ts_diff_median', 'action_ts_diff_median', 'user_name_

In [14]:
seeds = [2021]
oof = np.zeros(len(df_train))
importance = 0
fold_num = 10
pred_y = pd.DataFrame()
for seed in seeds:
    print('############################', seed)
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[LABEL])):
        print('-----------', fold)
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, LABEL])
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, LABEL])
        model = lgb.train(params, train, valid_sets=val, num_boost_round=10000,
                          early_stopping_rounds=100, verbose_eval=100)

        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d'%(fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain')/fold_num


df_train['oof'] = oof
score = roc_auc_score(df_train[LABEL], df_train['oof'])
print(score)

feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
print(feats_importance.sort_values('importance', ascending=False)[:10])

sub = pd.read_csv('data/submit_example.csv')

pred_y = pred_y.mean(axis=1)
sub['ret'] = pred_y


############################ 2021
----------- 0


/home/gitpod/.pyenv/versions/3.8.13/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/gitpod/.pyenv/versions/3.8.13/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.513458
Early stopping, best iteration is:
[4]	valid_0's auc: 0.523987
----------- 1


/home/gitpod/.pyenv/versions/3.8.13/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/gitpod/.pyenv/versions/3.8.13/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.519644
[200]	valid_0's auc: 0.532107
[300]	valid_0's auc: 0.54085
[400]	valid_0's auc: 0.543618
Early stopping, best iteration is:
[357]	valid_0's auc: 0.544446
----------- 2


...

[100]	valid_0's auc: 0.523377
Early stopping, best iteration is:
[45]	valid_0's auc: 0.530554
----------- 8
Training until validation scores don't improve for 100 rounds


/home/gitpod/.pyenv/versions/3.8.13/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/gitpod/.pyenv/versions/3.8.13/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	valid_0's auc: 0.523952
[200]	valid_0's auc: 0.518142
Early stopping, best iteration is:
[108]	valid_0's auc: 0.52684
----------- 9


/home/gitpod/.pyenv/versions/3.8.13/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/gitpod/.pyenv/versions/3.8.13/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.523476
Early stopping, best iteration is:
[12]	valid_0's auc: 0.567051
0.5268267103131414
                            name   importance
23                      ts_diff2  1098.056300
22                       ts_diff  1022.904920
20                         op_ts   973.469409
19                       sec_cos   593.461085
21                       last_ts   553.488940
17                           sec   546.794510
18                       sec_sin   532.028447
50        user_name_ts_diff_skew   277.684437
0                      user_name   208.174667
31  bus_system_code_ts_diff_mean   193.076955


In [15]:
sub[['id', 'ret']].to_csv('./results/lgb_5285.csv', index=False)